In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from scipy.stats import randint

# 📌 1. Veri Ön İşleme Fonksiyonu (Daha Güçlü)
def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 📌 Sayısal ve Kategorik Sütunları Doğru Belirle
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    print(f"🔹 Kategorik Sütunlar: {categorical_cols}")
    print(f"🔹 Sayısal Sütunlar: {numerical_cols}")

    # 📌 Eksik Değerleri Kontrol Et
    missing_info = df.isnull().sum()
    print("📌 Eksik Değerler:")
    print(missing_info[missing_info > 0])

    # 📌 Eksik değerleri doldurma
    num_imputer = SimpleImputer(strategy="median")
    cat_imputer = SimpleImputer(strategy="most_frequent")

    if numerical_cols:
        df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

    if categorical_cols:
        df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols].astype(str))  # String formatına çevir

    print("✅ Eksik değerler dolduruldu!")

    # 📌 Kategorik değişkenleri encode et
    for col in categorical_cols:
        df[col] = LabelEncoder().fit_transform(df[col])

    print("✅ Kategorik değişkenler encode edildi!")

    # 📌 Train seti için hedef değişkeni ayır
    if is_train:
        y = df["rainfall"]
        X = df.drop(columns=["rainfall", "id"])
        return X, y
    else:
        test_ids = df["id"]
        X = df.drop(columns=["id"])
        return X, test_ids

# 📌 2. Hyperparameter Tuning ile En İyi Random Forest Modelini Bul
def train_optimized_random_forest(X_train, y_train):
    param_grid = {
        "n_estimators": randint(200, 1000),
        "max_depth": randint(5, 20),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 10),
        "max_features": ["sqrt", "log2"],
    }

    rf_model = RandomForestClassifier(random_state=42, class_weight="balanced", n_jobs=-1)

    search = RandomizedSearchCV(rf_model, param_grid, n_iter=10, scoring="roc_auc", cv=3, verbose=1, n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    best_model = search.best_estimator_
    print(f"✅ En iyi Random Forest Modeli bulundu: {search.best_params_}")
    return best_model

# 📌 3. Tahmin Yapma ve Sonucu Kaydetme
def predict_and_save(model, test_file, output_file="submission.csv"):
    X_test, test_ids = preprocess_data(test_file, is_train=False)
    y_pred = model.predict_proba(X_test)[:, 1]  # Yağmur olma olasılığını al
    submission = pd.DataFrame({"id": test_ids, "rainfall": y_pred})
    submission.to_csv(output_file, index=False)
    print(f"✅ Tahminler {output_file} dosyasına kaydedildi!")

# 📌 Ana Akış
if __name__ == "__main__":
    train_file = "train.csv"
    test_file = "test.csv"

    # 📌 Veriyi yükle ve işle
    X_train, y_train = preprocess_data(train_file, is_train=True)

    # 📌 Modeli eğit (Hyperparameter Tuning ile)
    best_rf_model = train_optimized_random_forest(X_train, y_train)

    # 📌 Test verisi üzerinde tahmin yap ve sonucu kaydet
    predict_and_save(best_rf_model, test_file)


✅ Veri yüklendi: (2190, 13)


ValueError: at least one array or dtype is required